In [ ]:
#import the needed packages
import pandas as pd
import numpy as np
import os

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D
import seaborn as sns
import colorcet

import itertools
import re

from scipy import interpolate, stats, optimize
from statsmodels.stats.multitest import multipletests
import mathieu as mh
import importlib

In [ ]:
importlib.reload(mh)

In [ ]:
#import sys
#sys.path.insert(0, '/home/heasleylab/Lydia/')
### NO MORE NECESSARY. I moved mathieu.py to a directory added to the system PYTHONPATH (~/python_include)

In [ ]:
base_dir = '/home/heasleylab/Lydia/monosome_fitness_GCs/'
fig_path = f'{base_dir}fig/'

In [ ]:
dat_path = f'{base_dir}data/061025_LRH_UTF8.txt' # txt file from plate reader
sm_path = f'{base_dir}script/061025_straininfo.csv' #samples file .csv
ds = 'monosome_fitness'

sm, gc_dat = mh.parse_gc(dat_path, sm_path, ds, skiprows=11, blank_correct=False, Tend=24)


# Run logistic fit
#gc_logistic = mh.compute_gc_logistic(gc_dat, sm, ds, od='OD600')

# Run sliding window max slope fit
gc_maxslope, gc_maxslope_mean = mh.compute_gc_maxslope(gc_dat, sm, ds, wdw=12, average_reps=['background', 'growth_conditions'])

In [ ]:
#gc_maxslope_mean.to_csv('/home/heasleylab/Lydia/monosome_fitness_GCs/fig/05212025maxslope_means.csv', index=False)
gc_maxslope.to_csv('/home/heasleylab/Lydia/monosome_fitness_GCs/fig/06102025maxslope_all_24hrs.csv', index=False)

## Plots of the max slope fit

In [ ]:
#Plots of Replicates Merged by condition

In [ ]:
#Chat GPT version -- 36hrs in 2%dex
# Create shared x-axis for fit line
X = np.linspace(0, 36, 96)

# Predefine color mapping by replicate
rep_color = dict(zip(sm['replicate'].unique(), colorcet.glasbey_warm))

# Axes mapping for condition (only FOA_2.0dex used here)
cond_ax = dict(zip(['FOA_2.0dex', 'FOA_0.5dex', 'FOA_DMSO', 'FOA_1.0raf', 'FOA_2.5raf'], [0, 1, 2, 3, 4]))

# Filter for just one condition for plotting
condition = 'FOA_2.0dex'
filtered = gc_maxslope[gc_maxslope['growth_conditions'] == condition]

# Specify desired order explicitly
desired_order = ['parent', 'chr1', 'chr3', 'chr4', 'chr5', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr15']

# Get unique chromosomes to determine layout
chromosomes = [chrom for chrom in desired_order if chrom in filtered['chromosome'].unique()]
n_chroms = len(chromosomes)

# Create one column per chromosome
#fig, axes = plt.subplots(nrows=1, ncols=n_chroms, figsize=(10, 2 * n_chroms), sharex=True)

fig, axes = plt.subplots(nrows=1, ncols=n_chroms, figsize=(1.5 * n_chroms, 2))


# Handle single axis case
if n_chroms == 1:
    axes = [axes]

# Pre-index for speed
gc_maxslope_indexed = gc_maxslope.set_index('well')
gc_dat_indexed = gc_dat.set_index('well')

for ax, chrom in zip(axes, chromosomes):
    df_chrom = filtered[filtered['chromosome'] == chrom]

# Set plot limits, titles, axes
    
    ax.set_title(f'{condition}: {chrom}')
    ax.set_ylim(0, 1.5)
    ax.set_xlim(0, 36)
    ax.set_ylabel('OD600 (a.u.)', fontsize=8)
    ax.set_xlabel('time (hrs)', fontsize=8)
    ax.set_xticks(np.arange(0, 36, 10))   
    ax.set_yticks(np.arange(0, 1.5, 0.2))
    ax.tick_params(axis='both', labelsize=8)
    
    for (well, rep), df2 in df_chrom.groupby(['well', 'replicate']):
        # Get corresponding full time/OD data
        if well not in gc_dat_indexed.index:
            continue  # skip if well missing in data
        
        dat = gc_dat_indexed.loc[well]
        slope, intercept, wdmid = gc_maxslope_indexed.loc[well, ['slope', 'intercept', 'wdmid']]
        
        color = 'black'
        
        ax.plot(dat['time'], dat['OD600'], color=color, lw=.35, label=f'Rep {rep}')
        y = slope * X + intercept
        ax.plot(X, y, color=color, lw=0.25)
        ax.axvline(wdmid, ls=(0, (5, 5)), lw=0.25, color=color)
        
    
    #ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()
#fig.savefig('/home/heasleylab/Lydia/monosome_fitness_GCs/fig/growth_curves_maxslope-fit_replicates-together_061025_25hrs_2dex.svg')

In [ ]:
#Chat GPT version -- 36hrs in 0.5% dex
# Create shared x-axis for fit line
X = np.linspace(0, 36, 96)

# Predefine color mapping by replicate
rep_color = dict(zip(sm['replicate'].unique(), colorcet.glasbey_warm))

# Axes mapping for condition (only FOA_2.0dex used here)
cond_ax = dict(zip(['FOA_2.0dex', 'FOA_0.5dex', 'FOA_DMSO', 'FOA_1.0raf', 'FOA_2.5raf'], [0, 1, 2, 3, 4]))

# Filter for just one condition for plotting
condition = 'FOA_0.5dex'
filtered = gc_maxslope[gc_maxslope['growth_conditions'] == condition]

# Specify desired order explicitly
desired_order = ['parent', 'chr1', 'chr3', 'chr4', 'chr5', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr15']

# Get unique chromosomes to determine layout
chromosomes = [chrom for chrom in desired_order if chrom in filtered['chromosome'].unique()]
n_chroms = len(chromosomes)

# Create one column per chromosome
#fig, axes = plt.subplots(nrows=1, ncols=n_chroms, figsize=(10, 2 * n_chroms), sharex=True)

fig, axes = plt.subplots(nrows=1, ncols=n_chroms, figsize=(1.5 * n_chroms, 2))


# Handle single axis case
if n_chroms == 1:
    axes = [axes]

# Pre-index for speed
gc_maxslope_indexed = gc_maxslope.set_index('well')
gc_dat_indexed = gc_dat.set_index('well')

for ax, chrom in zip(axes, chromosomes):
    df_chrom = filtered[filtered['chromosome'] == chrom]

# Set plot limits, titles, axes
    
    ax.set_title(f'{condition}: {chrom}')
    ax.set_ylim(0, 0.8)
    ax.set_xlim(0, 36)
    ax.set_ylabel('OD600 (a.u.)', fontsize=8)
    ax.set_xlabel('time (hrs)', fontsize=8)
    ax.set_xticks(np.arange(0, 36, 10))   
    ax.set_yticks(np.arange(0, 0.8, 0.2))
    ax.tick_params(axis='both', labelsize=8)
    
    for (well, rep), df2 in df_chrom.groupby(['well', 'replicate']):
        # Get corresponding full time/OD data
        if well not in gc_dat_indexed.index:
            continue  # skip if well missing in data
        
        dat = gc_dat_indexed.loc[well]
        slope, intercept, wdmid = gc_maxslope_indexed.loc[well, ['slope', 'intercept', 'wdmid']]
        
        color = 'black'
        
        ax.plot(dat['time'], dat['OD600'], color=color, lw=.35, label=f'Rep {rep}')
        y = slope * X + intercept
        ax.plot(X, y, color=color, lw=0.25)
        ax.axvline(wdmid, ls=(0, (5, 5)), lw=0.25, color=color)
        
    
    #ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()
#fig.savefig('/home/heasleylab/Lydia/monosome_fitness_GCs/fig/growth_curves_maxslope-fit_replicates-together_061025_25hrs_05dex.svg')

In [ ]:
X = np.linspace(0, 36, 96)
rep_color = dict(zip(sm['replicate'].unique(), colorcet.glasbey_warm))
#cond_ax = dict(zip( ['ypd','uraDO','FOA','SD-complete'], [0,1,2,3]))
rep_ax = dict(zip([1,2,3,4,5,6,7], [0,1,2,3,4,5,6]))


for (bg,cond), df in gc_dat.loc[gc_dat['growth_conditions']=='FOA_2.0dex'].groupby(['background','growth_conditions']): #'chromosome' was taken out here

    fig, axes = plt.subplots(figsize=[16,4], ncols=7)
    fig.text(0.5, 0.95, f'{bg} in {cond}', size=14)
    
    for (w, rep), df1 in df.groupby(['well', 'replicate']):
        ax = axes[rep_ax[rep]]
        sns.scatterplot(x='time', y='OD600', hue='replicate', palette=rep_color, size=2, data=df1, ax=ax, legend=False, zorder=0)

        slope, intercept, wdmid = gc_maxslope.set_index('well').loc[w, ['slope', 'intercept', 'wdmid']]
        y = np.apply_along_axis(lambda x: x*slope+intercept, 0, X)
        ax.plot(X, y, c=rep_color[rep])
        ax.axvline(wdmid, ls='--', lw=0.5, c=rep_color[rep])
        
        ax.set_title(f'rep{rep:.0f} well {w}')
        ax.set_ylim(0,1.5)

 # Generate unique filename for this plot
    safe_bg = str(bg).replace(' ', '_')
    safe_cond = str(cond).replace(' ', '_')
    filename = f"growthcurve_{safe_bg}_{safe_cond}.svg"
    filepath = os.path.join(fig_path, filename)

    plt.show()
    
    # Save the full figure (only once!)
    #fig.savefig(filepath, format='svg', dpi=300, bbox_inches='tight')
    #print(f"Saved: {filepath}")
    
    plt.close()

In [ ]:
X = np.linspace(0, 36, 96)
rep_color = dict(zip(sm['replicate'].unique(), colorcet.glasbey_warm))
cond_ax = dict(zip(['FOA_2.0dex', 'FOA_0.5dex', 'FOA_DMSO', 'FOA_1.0raf', 'FOA_2.5raf'], [0, 1, 2, 3, 4]))

for chrom, df in gc_maxslope.loc[gc_maxslope['growth_conditions']=='FOA_2.0dex'].groupby('chromosome'):
    
    fig, axes = plt.subplots(ncols=2, figsize=[10,5])

    for (chrom, cond), df1 in df.groupby(['chromosome', 'growth_conditions']):
        
        ax = axes[cond_ax.get(cond, 0)]
        ax.set_title(f'{cond}: {chrom}')
        ax.set_ylim(0, 1.2)

        for (w, rep), df2 in df1.groupby(['well', 'replicate']):
            
            dat = gc_dat.set_index('well').loc[w]
            slope, intercept, wdmid = gc_maxslope.set_index('well').loc[w, ['slope', 'intercept', 'wdmid']]
            
            ax.plot(dat['time'], dat['OD600'], color=rep_color[rep], lw=2)
            y = np.apply_along_axis(lambda x: x*slope+intercept, 0, X)
            ax.plot(X, y, c=rep_color[rep], lw=0.5)
            ax.axvline(wdmid, ls=(0, (5,5)), lw=0.5, c=rep_color[rep])

    #plt.savefig(f'{fig_path}growth_curves_maxslope-fit_replicates-together_{chrom}.svg')
    plt.show()
    plt.close()

## Build table of relative max slope fits for the monosomes induction experiment

In [ ]:
rel_slope_monosomes = gc_maxslope.loc[gc_maxslope['growth_conditions']=='FOA'].copy()

for bg, df in dat.groupby('background'):
    if bg != 'LRH794':
        chrom_num = re.match('cen(\d{1,2})([ab]{0,1})', bg).group(1)
        transformant = re.match('cen(\d{1,2})([ab]{0,1})', bg).group(2)
        if transformant == '':
            transformant = 'a'
        rel_slope_monosomes.loc[df.index, 'monosome'] = f'chr{chrom_num}'
        rel_slope_monosomes.loc[df.index, 'transformant'] = f'{transformant}'
    else:
        rel_slope_monosomes.loc[df.index, 'monosome'] = 'none'
        rel_slope_monosomes.loc[df.index, 'transformant'] = 'none'

no_monosome_slope = rel_slope_monosomes.set_index('background').loc['LRH794', 'slope'].mean()
rel_slope_monosomes['relative_slope'] = rel_slope_monosomes['slope']/no_monosome_slope

In [ ]:
rel_slope_monosomes['monosome'].value_counts()
monosomes_order = ['none'] + [f'chr{i}' for i in [1,2,3,4,5,6,8,9,10,11,12,13,15]]
transformant_palette = dict(zip(['a', 'b', 'none'], colorcet.glasbey_bw))

fig, ax = plt.subplots(figsize=[16, 5])

sns.boxplot(x='monosome', y='relative_slope', hue='transformant', palette=dict(zip(['a', 'b', 'none'], ['w','w','w'])),
            fliersize=0, linewidth=1.5, order=monosomes_order, data=rel_slope_monosomes, ax=ax, legend=False)
sns.stripplot(x='monosome', y='relative_slope', hue='transformant', palette=transformant_palette, dodge=True,
              order=monosomes_order, data=rel_slope_monosomes, ax=ax)